In [1]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [ ]:
#не выполнять, если лэйблы проставлены, тест и трейн готовы
from sklearn import preprocessing
df = pd.read_csv("../data/mergedcleared1307.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.fit(df['id'])
df['id'] = Label_encoder.fit_transform(df['id'])
df.columns = ['labels', 'description']
df.to_csv('/home/sincosxy/rab/dataset.csv', index=False)
np.save('/home/sincosxy/rab/classes.npy', Label_encoder.classes_)
df = df.sample(frac=1).reset_index(drop=True)
train = df[:int(len(df)*0.8)]
test =  df[int(len(df)*0.8):]
train.to_csv('/home/sincosxy/rab/train.csv', index=False)
test.to_csv('/home/sincosxy/rab/test.csv', index=False)

In [2]:
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files={'train': '/home/sincosxy/rab/train.csv', 'test': '/home/sincosxy/rab/test.csv'})
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=100, padding='max_length'), batched=True)
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  # 
dataset = dataset.remove_columns(set(dataset['train'].features.keys()) - set(pytorch_style_columns))
set(dataset['train'].features.keys())
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda') 
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=1)
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=1)


Found cached dataset csv (/home/sincosxy/.cache/huggingface/datasets/csv/default-c45fd002eecbe8f6/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/sincosxy/.cache/huggingface/datasets/csv/default-c45fd002eecbe8f6/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-33ea085ce5b04645.arrow
Loading cached processed dataset at /home/sincosxy/.cache/huggingface/datasets/csv/default-c45fd002eecbe8f6/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-47070ad088e4822e.arrow


In [ ]:
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=12)
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=12)

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"

model = AutoModel.from_pretrained(model_name)#, num_labels=1024)
#попробуем позже

In [3]:
#from transformers import BertModel
class CustomBERTModel(nn.Module):
    def __init__(self):
        super(CustomBERTModel, self).__init__()
        self.bert = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
        #BertModel.from_pretrained("dbmdz/bert-base-italian-xxl-cased")
          ### New layers:
        self.linear1 = nn.Linear(1024, 8192)
        self.linear2 = nn.Linear(8192, 2648)
        #self.linear3 = nn.Linear(8192, 8192)
        #self.linear4 = nn.Linear(8192, 8192)
        #self.linear5 = nn.Linear(8192, 2648)
        ## 3 is the number of classes in this example
    def forward(self, ids, mask):
        #sequence_output, pooled_output = self.bert(ids, attention_mask=mask)
          # sequence_output has the following shape: (batch_size, sequence_length, 768)
        linear1_output = self.linear1(self.bert(ids, attention_mask=mask)[0][:,0,:].view(-1,1024)) ## extract the 1st token's embeddings
        #linear1_output = self.linear1(sequence_output)
        linear2_output = self.linear2(linear1_output)
        #linear3_output = self.linear3(linear2_output)
        #linear4_output = self.linear4(linear3_output)
        #linear5_output = self.linear5(linear4_output)
        return linear2_output

In [4]:
#tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-xxl-cased")
model = CustomBERTModel() # You can pass the parameters if required to have more flexible model
model.to(device)
#model.to(torch.device("cpu")) ## can be gpu
criterion = nn.CrossEntropyLoss() ## If required define your own criterion
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))       

In [ ]:
model.to(device)

In [ ]:
batch1 = next(iter(train_dataloader))
batch1

In [ ]:

outputs = model(batch1['input_ids'].to(torch.device('cpu')), batch1['attention_mask'].to(torch.device('cpu')))

In [ ]:
outputs.shape

In [ ]:
outputs[0][:,0,:]

In [ ]:
outputs[0]

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for batch in tqdm(train_dataloader): ## If you have a DataLoader()  object to get the data.
        #batch = {k: v.to(device) for k, v in batch.items()}
        #outputs = model(**batch)
        #data = batch[0]
        #targets = batch[1] ## assuming that data loader returns a tuple of data and its targets
        
        optimizer.zero_grad()   
        outputs = model(batch['input_ids'], batch['attention_mask'])
        #encoding = tokenizer.batch_encode_plus(data, return_tensors='pt', padding=True, truncation=True,max_length=50, add_special_tokens = True)
        #outputs = model(input_ids, attention_mask=attention_mask)
        outputs =  torch.nn.functional.softmax(outputs, dim=1)#F.log_softmax(outputs, dim=1)
        # torch.nn.functional.
        #input_ids = encoding['input_ids']
        #attention_mask = encoding['attention_mask']
        loss = criterion(outputs, batch['labels'])
        loss.backward()
        optimizer.step()

 79%|████████████████████████████████▎        | 146665/186380 [8:08:47<2:12:01,  5.01it/s]

In [ ]:

optimizer = AdamW(model.parameters(), lr=2e-6)

num_epochs = 15

num_training_steps = num_epochs * len(train_dataloader)


lr_scheduler = get_scheduler(
    "constant_with_warmup",
    optimizer=optimizer,
    num_warmup_steps=2000,
    num_training_steps=num_training_steps
)

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
from tqdm import tqdm

In [ ]:
#next(iter(train_dataloader))
model.to(device)

In [ ]:
num_epochs = 10

In [ ]:
best_f1 = 0.
show_train_loss_every_num_epoch = 0.1

for epoch in range(num_epochs):
    print(40*'-', '\nepoch', epoch+1)
    model.train()
    losses = []

    for i, batch in enumerate(tqdm(train_dataloader)):
        model.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        losses.append(loss.item())

        #f i%int(len(train_dataloader)*show_train_loss_every_num_epoch)==int(len(train_dataloader)*show_train_loss_every_num_epoch)-1:
    print(f'train loss [{i*100/len(train_dataloader):.2f}%]: {np.array(losses).mean():.3f}')
    losses = []
    print('\nvalidating')

    f1 = load_metric('f1')
    acc = load_metric('accuracy')
    precision = load_metric('precision')
    recall = load_metric('recall')
    with torch.no_grad():
        model.eval()
                
        for batch in tqdm(test_dataloader):
        ##for batch in test_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            f1.add_batch(predictions=predictions, references=batch["labels"])    
            acc.add_batch(predictions=predictions, references=batch["labels"])    
            precision.add_batch(predictions=predictions, references=batch["labels"])    
            recall.add_batch(predictions=predictions, references=batch["labels"])    

        print('weighted summary:')
        print('Test acc:', acc.compute()['accuracy'])
        print('Test precision:', precision.compute(average = 'weighted')['precision'])
        print('Test recall:', recall.compute(average = 'weighted')['recall'])
        f1_weighted = f1.compute(average = 'weighted')['f1']
        print('Test f1:', f1_weighted, '\n')

        if f1_weighted > best_f1:
            best_f1 = f1_weighted
            model.save_pretrained("../../best_model2705")

    model.train()

In [ ]:
from sklearn import preprocessing
from sklearn.metrics import classification_report
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./classes.npy', allow_pickle=True)

true = []
preds = []

model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()

print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

In [ ]:
inputs = tokenizer('покрышки', truncation = True, max_length=100, padding='max_length', return_tensors="pt")
#batch = {k: torch.tensor(v).to(device) for k, v in txt.items()}
model.to(torch.device('cpu'))
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
#Label_encoder.inverse_transform(predicted_class_id)
a = []
a.append(predicted_class_id)

In [ ]:
Label_encoder.inverse_transform([a])

In [ ]:
def predict_class(text):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        return Label_encoder.inverse_transform([predicted_class_id])


In [ ]:
print(predict_class("Велосипед"), predict_class("велосипед"), predict_class("Велосипедов"))